Import library yang dipakai

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tweepy
import json
from tweepy import OAuthHandler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam

Baca dataset

In [ ]:
df = pd.read_csv('newdataset.csv', sep=';')
tweets = df['Tweet'].astype(str)
label = df['label']
df.head()

,Tweet,label
0,?? ????? ?????? ???????? ???????? ????-????-??...,0
1,The clocks go back 1 hour tonight! An extra ho...,0
2,#blackpink #blackpinkfanart #kpop #kimjennie #...,0
3,BLACKPINK-DDU DU DDU DU Which is your favourit...,0
4,@Blackipinkoffi1. @Blackipinkoffi1 @Blackipink...,0


In [ ]:
print(len(tweets))
print(len(label))

30625
30625


In [ ]:
df = df.sample(frac=1)
df.head()

,Tweet,label
23592,udah dibilang skrg tu mana aja sama pendidikan...,1
1244,@Saham_fess Ipo aja belom wkwkwk,0
9282,#CriticaInternacional Mirada CrÃ­tica: Nicarag...,0
22850,Polisi 24 jam jaga penyekatan mudik https://...,1
7844,Gempa Majene: 3 Warga Meninggal Dunia. 24 Luka...,0


Inisiasi Tokenizer dan padded sequence

In [ ]:
VOCAB_SIZE = 10000
OOV = '<OOV>'
PADDING = 'pre'
TRUNCATING = 'post'
MAXLEN = 125
EMBEDDING = 750

In [ ]:
tokenizer = Tokenizer(num_words = VOCAB_SIZE, oov_token = OOV)
tokenizer.fit_on_texts(tweets)
word_index = tokenizer.word_index
print(len(word_index))
print(word_index)

77835
{'<OOV>': 1, 't': 2, 'co': 3, 'https': 4, 'di': 5, 'dan': 6, 'yang': 7, 'yg': 8, 'ini': 9, 'jokowi': 10, 'pak': 11, 'gempa': 12, 'untuk': 13, 'teaserquest': 14, 'kerja': 15, 'ada': 16, 'uu': 17, 'tolakrevisiuukpk': 18, 'mudik': 19, 'ppkm': 20, 'indonesia': 21, 'penyekatan': 22, 'preman': 23, 'bencanaalam': 24, 'pensiun': 25, 'cipta': 26, 'dari': 27, 'palestina': 28, 'saham': 29, 'itu': 30, 'exo': 31, 'ya': 32, 'dengan': 33, 'hidupmahasiswa': 34, 'jadi': 35, 'bisa': 36, 'lagi': 37, 'ke': 38, 'mikro': 39, 'ikatancinta': 40, 'the': 41, 'tolakruukuhp': 42, 'dalam': 43, '2021': 44, 'sekolahonline': 45, 'komisaris': 46, 'ga': 47, 'weareoneexo': 48, '5': 49, 'de': 50, 'bencana': 51, 'tidak': 52, 'saya': 53, 'juga': 54, 'aja': 55, 'sama': 56, 'kita': 57, 'apa': 58, 'mau': 59, 'covid': 60, 'aku': 61, '19': 62, 'gak': 63, 'tapi': 64, 'akan': 65, 'fight': 66, 'a': 67, 'pada': 68, 'feeling': 69, 'dont': 70, 'atau': 71, 'hari': 72, 'tni': 73, 'kalo': 74, 'kpk': 75, 'warga': 76, 'presiden': 77

In [ ]:
sequences = tokenizer.texts_to_sequences(tweets)

In [ ]:
print(tweets[30000])
print(sequences[30000])

Pak Jokowi presiden palingg mboiisss. pembangunan infrastruktur merata  Plisss Komut PLN pak @jokowi  ðŸ™ðŸ»
[11, 10, 77, 1, 1, 1266, 1776, 3182, 1, 1170, 493, 11, 10, 1594]


In [ ]:
padded_sequences = pad_sequences(sequences, padding=PADDING, truncating=TRUNCATING, maxlen=MAXLEN)
print(padded_sequences[30000])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0   11
   10   77    1    1 1266 1776 3182    1 1170  493   11   10 1594]


Mengeluarkan pickle dari padded sebelumnya

In [ ]:
import pickle

with open('./tokenizerkebijakan.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
train_len = (len(padded_sequences) - 1000)

train_data = padded_sequences[:train_len]
train_label = label[:train_len]
val_data = padded_sequences[train_len:]
val_label = label[train_len:]

print(len(train_data))
print(len(train_label))
print(len(val_data))
print(len(val_label))

29625
29625
1000
1000


In [ ]:
arr_td = np.array(train_data)
arr_tl = np.array(train_label)
arr_vd = np.array(val_data)
arr_vl = np.array(val_label)

print(train_data.shape)
print(train_label.shape)
print(val_data.shape)
print(val_label.shape)

(29625, 125)
(29625,)
(1000, 125)
(1000,)


In [ ]:
model = Sequential([
                    Embedding(VOCAB_SIZE, EMBEDDING, input_length = MAXLEN),
                    Bidirectional(
                        LSTM(125, dropout=0.2)),
                    Dense(500, activation='relu'),
                    Dropout(0.5),
                    Dense(1, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

history = model.fit(
    train_data, train_label,
    epochs = 10,
    validation_data = (val_data, val_label),
    verbose = 1
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 125, 750)          7500000   
_________________________________________________________________
bidirectional (Bidirectional (None, 250)               876000    
_________________________________________________________________
dense (Dense)                (None, 500)               125500    
_________________________________________________________________
dropout (Dropout)            (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 501       
Total params: 8,502,001
Trainable params: 8,502,001
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
926/926 [==============================] - 103s 101ms/step - loss: 0.0652 - acc: 0.9762 - val_loss: 0

In [ ]:
saved_model_path = './model_policy.h5'
model.save(saved_model_path)

Pengetesan model

In [ ]:
consumer_key = "i2Y3uMSk9tkm3e241eKLPCcZb"
consumer_secret = "jKRjdJVxB6sU65u79oXRCSdZVCjTn2YU7WrS88cV3nRHtgQ2Hx"
access_key = "1356544029874262016-KTJqhY6dkIFbyEbKRjqdvTEAn4A0QG"
access_secret = "C1JAb7h2VVGtMfXDZuHMi5Kxotcu8dfjLQILL5JobufkD"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
INDONESIA_WOE_ID = 23424846
indonesia_trends = api.trends_place(INDONESIA_WOE_ID)

print(indonesia_trends)

[{'trends': [{'name': '#BisaGilaGrabFood', 'url': 'http://twitter.com/search?q=%23BisaGilaGrabFood', 'promoted_content': None, 'query': '%23BisaGilaGrabFood', 'tweet_volume': None}, {'name': '#EXO_DFTFOutNow', 'url': 'http://twitter.com/search?q=%23EXO_DFTFOutNow', 'promoted_content': None, 'query': '%23EXO_DFTFOutNow', 'tweet_volume': 1809799}, {'name': '#ToEXOPlanetAndBeyond', 'url': 'http://twitter.com/search?q=%23ToEXOPlanetAndBeyond', 'promoted_content': None, 'query': '%23ToEXOPlanetAndBeyond', 'tweet_volume': 2104100}, {'name': '#IkatanCintaEp306', 'url': 'http://twitter.com/search?q=%23IkatanCintaEp306', 'promoted_content': None, 'query': '%23IkatanCintaEp306', 'tweet_volume': None}, {'name': '#DONT_FIGHT_THE_FEELING', 'url': 'http://twitter.com/search?q=%23DONT_FIGHT_THE_FEELING', 'promoted_content': None, 'query': '%23DONT_FIGHT_THE_FEELING', 'tweet_volume': 3386188}, {'name': 'DITA DITAMIN DAY', 'url': 'http://twitter.com/search?q=%22DITA+DITAMIN+DAY%22', 'promoted_content':

In [ ]:
def Encoding(text):
    with open('./tokenizerkebijakan.pickle', 'rb') as handle:
        tokenizer_new = pickle.load(handle)
    sequence = tokenizer_new.texts_to_sequences([text])
    paded = pad_sequences(sequence, padding=PADDING, truncating=TRUNCATING, maxlen=MAXLEN)
    
    return paded

In [ ]:
def TextProcessing_Policy(text, model3):
    sequence = Encoding(text)
    result = model.predict(sequence)
    
    return result[0][0] * 100

In [ ]:
for trends in indonesia_trends[0]['trends']:
  if TextProcessing_Policy(str(trends["name"]), model) > 50:
    search = [str(trends["name"])]
    tweets = tweepy.Cursor(api.search, q=search, lang="in").items(50)
    for tweet in tweets:
      if (not tweet.retweeted) and ('RT @' not in tweet.text):
        if TextProcessing_Policy(str(tweet.text), model) > 70:
          print(json.loads(json.dumps(search)))
          print(json.loads(json.dumps(tweet.text, indent=4)))
          print("Termasuk kebijakan karena : {:.2f}%".format(TextProcessing_Policy(str(tweet.text), model)))

['BHS INGGRIS']
@cookieshoba Bhs. Inggris
Termasuk kebijakan karena : 93.46%
['BHS INGGRIS']
bhs inggris kayanya? https://t.co/Ah6kKv32nY
Termasuk kebijakan karena : 97.20%
['BHS INGGRIS']
@convomf Ngrangkum
Hafalan
Istirahat
Bhs inggris
Guru agama
Temen pinter
Termasuk kebijakan karena : 99.83%
['BHS INGGRIS']
Dulu saya ngefans sama Emha.
Setelah saya lihat di youtube dia mengolok oleh Jokowi krn tak puntar bhs inggris saat… https://t.co/Nz9Qj5HWAO
Termasuk kebijakan karena : 100.00%
['BHS INGGRIS']
@convomf ngerangkum
hafalan
jamkos
bhs inggris
guru agama
temen ngakak
Termasuk kebijakan karena : 99.93%
['BHS INGGRIS']
@convomf 1. NGERANGKUM
2. HITUNGAN
3. JAMKOS
4. BHS INGGRIS
5. GURU AGAMA
6. TEMEN NGAKAK
Termasuk kebijakan karena : 99.65%
['BHS INGGRIS']
@convomf 1. NGERANGKUM
2. HAFALAN
3. JAMKOS
4. BHS INGGRIS
5. GURU BK
6. TEMEN NGAKAK
Termasuk kebijakan karena : 99.91%
['BHS INGGRIS']
@convomf presentasi
hafalan
jamkos
bhs inggris
guru bk 
temen ngakak
Termasuk kebijakan karena

In [ ]:
#model3 = Sequential([
                     Embedding(VOCAB_SIZE, EMBEDDING, input_length= MAXLEN),
                     GlobalAveragePooling1D(),
                     Dense(125, activation= 'relu'),
                     Dense(25, activation= 'relu'),
                     Dense(5, activation= 'relu'),
                     Dense(1, activation= 'sigmoid')
])

model3.summary()

model3.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

history3 = model3.fit(
    train_data, train_label,
    epochs = 30,
    validation_data = (val_data, val_label),
    verbose = 1
)

IndentationError: ignored